In [33]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import base64
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalShelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "bee"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard Abiel A. Zamora'))),
    html.Hr(),
    html.Div(
        #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        className='row',
        style={'display': 'flex'},
        children=[
            html.Button(id='button-one', n_clicks=0, children='Water Resc.'),
            html.Button(id='button-two', n_clicks=0, children='Mountain Or Wilderness Resc.'),
            html.Button(id='button-three', n_clicks=0, children='Disaster Resc. Or Ind. Tracking'),
            html.Button(id='button-four', n_clicks=0, children='reset')
        ]
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
	#If you completed the Module Six Assignment, you can copy in the code you created here 
        page_size=20,  # set number of rows per page
        sort_action='native',  # enable sorting
        filter_action='native',  # enable filtering
        style_data_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)'}],  # alternate row colors
    ),
    html.Div(id='map-id')
    ,
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display' : 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',

                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)

@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    ### FIX ME Add code to filter interactive data table with MongoDB queries
    query = {}
    if filter_type == 'Water Rescue':
        query = {'animal_type': 'Dog', 'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']}, 'sex_upon_outcome': 'Intact Female', '$or': [{'age_upon_outcome_in_weeks': {'$gte': 26}}, {'age_upon_outcome_in_weeks': {'$lt': 156}}]}
    elif filter_type == 'Mountain/Wilderness Rescue':
        query = {'animal_type': 'Dog', 'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']}, 'sex_upon_outcome': 'Intact Male', '$or': [{'age_upon_outcome_in_weeks': {'$gte': 26}}, {'age_upon_outcome_in_weeks': {'$lt': 156}}]}
    elif filter_type == 'Disaster Rescue/Individual Tracking':
        query = {'animal_type': 'Dog', 'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']}, 'sex_upon_outcome': 'Intact Male', '$or': [{'age_upon_outcome_in_weeks': {'$gte': 20}}, {'age_upon_outcome_in_weeks': {'$lt': 300}}]}
    else:
        query = {}    
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    return (data,columns)

@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    ### FIX ME Add code to filter interactive data table with MongoDB queries
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    return (data,columns)


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{'if': { 'column_id': i },'background_color': '#D2F3FF'} for i in selected_columns]


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]
    if viewData is None:
        return {}

    df = pd.DataFrame(viewData)
    fig = px.pie(df, names='breed', title='% of breeds')
    return [dcc.Graph(figure=fig)]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    #FIXME: Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [ dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], zoom=15, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
            dl.Tooltip(dff['breed']),        
            dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.loc[0,'name'])
            ])
        ])         
    ]

SyntaxError: unexpected EOF while parsing (<ipython-input-33-65e7b721a2a0>, line 189)